In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [38]:
# Загрузка данных
data = pd.read_csv('/home/eva/project/predictive_real_estate_price_analytics_service/data/processed/progress_data.csv')
data.head()

,url_id,total_meters,price
0,316749660,38.9,13477999
1,315778606,32.0,18700000
2,311419733,34.1,19000000
3,315916888,32.9,8700000
4,306867070,41.1,33000000


In [39]:

# Стандартизация признака
#scaler = StandardScaler()
#data['total_meters'] = scaler.fit_transform(data[['total_meters']])

# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Коэффициенты модели
print(f"Коэффициент при площади: {model.coef_[0]:.2f}")
print(f"Свободный член: {model.intercept_:.2f}")


Среднеквадратичная ошибка (MSE): 6912767876366181.00
Корень из среднеквадратичной ошибки (RMSE): 83143056.69
Коэффициент детерминации R²: 0.555481
Средняя ошибка предсказания: 40234921.12 рублей
Коэффициент при площади: 1453565.12
Свободный член: -51745586.42


In [40]:
import joblib

# Сохранение модели
model_path = '/home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl'

# Исправлено: передаем модель и путь для сохранения
joblib.dump(model, model_path)
print(f"Модель сохранена в файл {model_path}")  # Исправлена f-строка

# Загрузка модели
loaded_model = joblib.load(model_path)
print("Модель загружена из файла")

Модель сохранена в файл /home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl
Модель загружена из файла
